## Ch.11.(2/2) Deep learning for text

Chapter contents:

* Transformer 아키텍처
* Sequence-to-sequence 학습


## Ch.11.3 The Transformer architecture

2017년부터 새로운 모델 아키텍처: Transformer (Attention is all you need)

"neural attention"이라는 간단한 메커니즘을 사용하여 순환(recurrent) 레이어나 컨볼루션(convolution) 레이어가 없는 강력한 시퀀스 모델을 만들 수 있습니다.

### Understanding self-attention

모델이 보는 모든 입력 정보가 당면한 작업에 똑같이 중요하지는 않습니다.

모델은 일부 특징에 "더 많은 주의를 기울이고(pay more attention)" 다른 특징에 "덜 주의를 기울여야(pay less attention)" 합니다.

비슷한 생각:

* convnet의 Max pooling은 공간 영역의 특징 풀을 살펴보고, 유지할 특징을 하나만 선택합니다.


* TF-IDF 정규화는 다른 토큰이 전달할 가능성이 있는 정보의 양에 따라 토큰에 중요도 점수를 할당합니다.

<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-05.png" width="300"><p style="text-align:center">Figure 11.5 The general concept of “attention” in deep learning: input features get assigned “attention scores,” which can be used to inform the next representation of the input. From raw text to vectors.</p>

Attention 메커니즘을 사용하여 특징을 문맥-인식(context-aware)이 가능하게 할 수 있습니다.

벡터 공간은 서로 다른 단어 간의 의미 관계의 "모양"을 캡처합니다.

한 단어는 고정된 위치를 갖는다

그러나, 단어의 의미는 일반적으로 문맥에 따라 다릅니다.

스마트 임베딩 공간: 단어를 둘러싼 다른 단어에 따라 단어에 대해 다른 벡터 표현 제공

self-attention의 목적은 시퀀스에서 관련 토큰의 표현을 사용하여 토큰의 표현을 변조하는 것입니다.

<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-06.png" width="500"><p style="text-align:center">Figure 11.6. Self-attention: attention scores are computed between “station” and every other word in the sequence, and they are then used to weight a sum of word vectors that becomes the new “station” vector.</p>

* Compute relevancy scores (dot product)
* Scale (1/sqrt) and softmax
* Weighted (scores) sum of vectors

In [ ]:
def self_attention(input_sequence):
    output = np.zeros(shape=input_sequence.shape)
    for i, pivot_vector in enumerate(input_sequence):
        scores = np.zeros(shape=(len(input_sequence),))
        for j, vector in enumerate(input_sequence):
            scores[j] = np.dot(pivot_vector, vector.T)
        scores /= np.sqrt(input_sequence.shape[1])
        scores = softmax(scores)
        new_pivot_representation = np.zeros(shape=pivot_vector.shape)
        for j, vector in enumerate(input_sequence):
            new_pivot_representation += vector * scores[j]
        output[i] = new_pivot_representation
    return output

#### Generalized self-attention: the query-key-value model

self-attention을 위한 백터화 구현
```
    num_heads = 4 
    embed_dim = 256 
    mha_layer = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
    outputs = mha_layer(inputs, inputs, inputs)
```
지금까지 단 하나의 시퀀스를 사용함. 
그러나 기계 번역용으로 개발된 Transformer (2개의 시퀀스: source, target)

<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-06-UN01.png" width="450">

* 입력(A)의 각 토큰에 대해, 
* 토큰이 입력(B)의 모든 토큰과 얼마나 관련되어 있는지 계산
* 이 점수를 사용하여 입력(C)의 토큰 합계에 가중치를 부여합니다.

<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-06-UN02.png" width="450">
 
<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-07.png" width="500"><p style="text-align:center">Figure 11.7 Retrieving images from a database: the “query” is compared to a set of “keys,” and the match scores are used to rank “values” (images).</p>

* Source: "How's the weather today?"
* Target: "¿Qué tiempo hace hoy?" 

* query: target
* keys: source
* values: source

### Multi-head attention

다중 헤드(Multi-head): self-attention layer의 출력 공간은 개별적으로 학습된 독립적인 부분 공간 세트로 나누어집니다.

초기 쿼리(query), 키(key) 및 값(value)은 3개의 독립적인 dense projections 세트를 통해 전송되어 3개의 개별 벡터가 생성됩니다.

각 벡터는 neural attention 을 통해 처리됩니다.

3개의 출력은 다시 하나의 출력 시퀀스로 연결됩니다.

학습 가능한 dense projections의 존재는 레이어가 무언가를 학습할 수 있도록 합니다(순수한 상태없는 변환(stateless transformation)과 반대).

독립 헤드(Independent heads): 레이어가 각 토큰에 대한 다양한 특징 그룹을 학습하도록 돕습니다.

<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-08.png" width="450"><p style="text-align:center">Figure 11.8 The MultiHeadAttention layer</p>

<img src="https://production-media.paperswithcode.com/methods/multi-head-attention_l1A3G7a.png" width="200"><p style="text-align:center">Multi-head attention (Another perspective)</p>

### The Transformer encoder

Dense projections 작업 -> attention 메커니즘의 출력에도 추가함.

또한

* residual connection 추가(깊어지기 때문에)
* 정규화 추가(역전파 동안 더 나은 gradients 흐름을 위해)

<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-09.png" width="200"><p style="text-align:center">Figure 11.9 The TransformerEncoder chains a MultiHeadAttention layer with a dense projection and adds normalization as well as residual connections.</p>

**Getting the data**

In [ ]:
# !curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz
# !rm -r aclImdb/train/unsup

**Preparing the data**

In [0]:
# import os, pathlib, shutil, random
# from tensorflow import keras
# batch_size = 32
# base_dir = pathlib.Path("aclImdb")
# val_dir = base_dir / "val"
# train_dir = base_dir / "train"
# for category in ("neg", "pos"):
#     os.makedirs(val_dir / category)
#     files = os.listdir(train_dir / category)
#     random.Random(1337).shuffle(files)
#     num_val_samples = int(0.2 * len(files))
#     val_files = files[-num_val_samples:]
#     for fname in val_files:
#         shutil.move(train_dir / category / fname,
#                     val_dir / category / fname)

In [1]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**Vectorizing the data**

In [2]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [3]:
list(int_test_ds.take(1))[0]

(<tf.Tensor: shape=(32, 600), dtype=int64, numpy=
 array([[  15,  232,   15, ...,    0,    0,    0],
        [  89,   27, 4403, ...,    0,    0,    0],
        [  44,   23,   67, ...,    0,    0,    0],
        ...,
        [  10,   26, 4306, ...,    0,    0,    0],
        [ 463,    2, 1651, ...,    0,    0,    0],
        [ 356,   17,    8, ...,    0,    0,    0]], dtype=int64)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1])>)

**Transformer encoder implemented as a subclassed `Layer`**

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**Using the Transformer encoder for text classification**

In [5]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

**Training and evaluating the Transformer encoder based model**

In [6]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20
625/625 [==============================] - 35s 49ms/step - loss: 0.4706 - accuracy: 0.7812 - val_loss: 0.3787 - val_accuracy: 0.8334
Epoch 2/20
625/625 [==============================] - 30s 48ms/step - loss: 0.2997 - accuracy: 0.8753 - val_loss: 0.3208 - val_accuracy: 0.8628
Epoch 3/20
625/625 [==============================] - 30s 48ms/step - loss: 0.2353 - accuracy: 0.9056 - val_loss: 0.4505 - val_accuracy: 0.8248
Epoch 4/20
625/625 [==============================] - 30s 48ms/step - loss: 0.1866 - accuracy: 0.9290 - val_loss: 0.3171 - val_accuracy: 0.8792
Epoch 5/20
625/625 [==============================] - 30s 48ms/step - loss: 0.1514 - accuracy: 0.9423 - val_loss: 0.3410 - val_accuracy: 0.8914
Epoch 6/20
625/625 [==============================] - 30s 48ms/step - loss: 0.1284 - accuracy: 0.9507 - val_loss: 0.3877 - val_accuracy: 0.8832
Epoch 7/20
625/625 [==============================] - 31s 49ms/step - loss: 0.1092 - accuracy: 0.9596 - val_loss: 0.4322 - val_accuracy:

* binary_1gram: 0.887
* binary_2gram: 0.892
* tfidf_2gram: 0.897
* one_hot_bidir_lstm: 0.873
* embeddings_bidir_lstm: 0.868
* embeddings_bidir_lstm_with_masking: 0.872
* glove_embeddings_sequence model: 0.868
* transformer_encoder: 0.874

기본 모델보다 좋지 않습니다. 왜요?

현재 Transformer 모델: 시퀀스 모델이 아닙니다.

Attention 계층: 토큰을 집합(set)으로 봅니다.

토큰 순서 변경: 동일한 컨텍스트 인식 표현

<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-10.png" width="250"><p style="text-align:center">Figure 11.10 Features of different types of NLP models</p>

#### Using positional encoding to re-inject order information


위치 인코딩: 모델에 단어-순서 정보에 대한 액세스 부여

각 단어 임베딩에 단어의 위치 추가

워드 임베딩 구성요소:
* 단어 벡터(word vector)(독립 표현)
* 위치 벡터(position vector)(단어의 위치를 나타냄)

구현

모델은 위치-임베딩(position-embedding) 벡터를 학습합니다(단어 인덱스를 포함하는 것과 동일한 방식).

**Implementing positional embedding as a subclassed layer**

In [7]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

#### Putting it all together: A text-classification Transformer

**Combining the Transformer encoder with positional embedding**

In [8]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posit  (None, None, 256)        5273600   
 ionalEmbedding)                                                 
                                                                 
 transformer_encoder_1 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

* binary_1gram: 0.887
* binary_2gram: 0.892
* tfidf_2gram: 0.897
* one_hot_bidir_lstm: 0.873
* embeddings_bidir_lstm: 0.868
* embeddings_bidir_lstm_with_masking: 0.872
* glove_embeddings_sequence model: 0.868
* transformer_encoder: 0.874
* full_transformer_encoder: 0.882

Improved but still not as good as the Bag-of-words ones.

### When to use sequence models over bag-of-words models?

Bag-of-words는 구식이 아닙니다.

어떤 경우에는 bag-of-bigrams 위에 Dense layers의 작은 스택이 완벽하게 유효합니다.

다양한 유형의 텍스트 데이터 세트에 대한 다양한 텍스트 분류 기술 분석

중요 기준: 훈련 데이터의 샘플 수와 샘플당 평균 단어 수 간의 비율


<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-11.png" width="350"><p style="text-align:center">Figure 11.11 A simple heuristic for selecting a text-classification model: the ratio between the number of training samples and the mean number of words per sample</p>

시퀀스 모델은 다음과 같은 경우에 가장 잘 작동합니다.

* 많은 훈련 데이터를 사용할 수 있으면서
* 각 샘플은 상대적으로 짧은 경우

## Beyond text classification: Sequence-to-sequence learning

시퀀스-투-시퀀스(Sequence-to-sequence) 학습 응용

* 기계 번역
* 텍스트 요약(summarization)
* 질의응답
* 챗봇(Chatbot)
* 텍스트 생성 
* etc.

일반 템플릿:

  * 인코더: 소스 시퀀스를 중간 표현(intermediate representation)으로 바꿉니다.
  * 디코더: 다음의 두개를 보고 목표 시퀀스의 다음 토큰 i를 예측하도록 훈련됨
      * 이전 토큰(0 to i-1) 및
      * 인코딩된 소스 시퀀스

추론 과정: 목표 시퀀스에 액세스할 수 없습니다. 한 번에 하나의 토큰을 생성합니다.

1. 인코더에서 인코딩된 소스 시퀀스를 얻습니다.
2. 디코더는 다음으로 시작합니다.
   * 인코딩된 소스 시퀀스
   * 초기 "시드" 토큰 [start]
  
   시퀀스의 첫 번째 실제 토큰을 예측합니다.
  
3. 디코더로 피드백되는 예측 시퀀스(스톱 토큰 [end] 가 생성될 때까지)


<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-12.png" width="450"><p style="text-align:center">Figure 11.12 Sequence-to-sequence learning: the source sequence is processed by the encoder and is then sent to the decoder. The decoder looks at the target sequence so far and predicts the target sequence offset by one step in the future. During inference, we generate one target token at a time and feed it back into the decoder.</p>

<img src="https://www.guru99.com/images/1/111318_0848_seq2seqSequ1.png">

### A machine translation example

In [14]:
# !wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

In [16]:
text_file = "spa-eng/spa.txt"
with open(text_file, encoding='utf-8') as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [17]:
import random
print(random.choice(text_pairs))

('Tom waited patiently for Mary.', '[start] Tom esperó pacientemente a Mary. [end]')


In [18]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs   = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs  = text_pairs[num_train_samples + num_val_samples:]

**Vectorizing the English and Spanish text pairs**

2개의 TextVectorization 레이어 준비

일반적으로 [ 및 ] 문자가 제거됩니다만, 이번 예제에서는 [start] 및 [end]은 보존되어야 합니다.

In [19]:
import tensorflow as tf
import string
import re

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,     # for [start] or [end]
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

**Preparing datasets for the translation task**

튜플(tuple) 반환(inputs, target)

* 입력은 "encoder_inputs"(영어 문장) 및 "decoder_inputs"(스페인어 문장)의 두 키가 있는 dictionary 입니다.

* target은 한 단계 앞선 오프셋을 갖는 스페인어 문장 

In [20]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],    # [start], 1, 2. ..., n
    }, spa[:, 1:])                 # 1, 2. ..., n, [end]

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [21]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


### Sequence-to-sequence learning with RNNs

2015-2017년에는 시퀀스-투-시퀀스 학습에서 RNN이 지배적이었음 (2017년경 Google 번역 참조)

시퀀스를 시퀀스로 바꾸는 순진한 방법

```
inputs = keras.Input(shape=(sequence_length,), dtype='int64')
x = layers.Embedding(input_dim=vocab_size, output_dim=128)(inputs)
x = layers.LSTM(32, return_sequences=True)(x)         # See this return_sentences
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
```
<img src="https://miro.medium.com/max/1400/1*zeOG90tmye654fQjLslkQA.png" width="400" >

두 가지 주요 문제:
* 타겟 seq 길이는 소스 seq와 같아야 합니다.
* 모델은 타겟에서 토큰 N을 예측하기 위해 소스에서 토큰 0...N만 참고합니다.


적절한 방법:

* 전체 seq를 벡터(들)의 집합으로 인코딩
* N+1을 예측하기 위해 target seq에서 위의 것들과 함께 elemens 0...N을 제공합니다.

<br>

<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-13.png" width="350"><p style="text-align:center">Figure 11.13 A sequence-to-sequence RNN: an RNN encoder is used to produce a vector that encodes the entire source sequence, which is used as the initial state for an RNN decoder.</p>

**GRU-based encoder**

In [22]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile3.uf.tistory.com%2Fimage%2F99D1914F5B2917F136F0E5" width="400">

**GRU-based decoder and the end-to-end model**

In [23]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

**Training our recurrent sequence-to-sequence model**

In [24]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 [==============================] - 91s 64ms/step - loss: 1.6376 - accuracy: 0.4166 - val_loss: 1.3168 - val_accuracy: 0.5069
Epoch 2/15
1302/1302 [==============================] - 83s 64ms/step - loss: 1.3157 - accuracy: 0.5255 - val_loss: 1.1546 - val_accuracy: 0.5691
Epoch 3/15
1302/1302 [==============================] - 83s 64ms/step - loss: 1.1735 - accuracy: 0.5757 - val_loss: 1.0726 - val_accuracy: 0.6003
Epoch 4/15
1302/1302 [==============================] - 86s 66ms/step - loss: 1.0828 - accuracy: 0.6076 - val_loss: 1.0386 - val_accuracy: 0.6204
Epoch 5/15
1302/1302 [==============================] - 86s 66ms/step - loss: 1.0358 - accuracy: 0.6326 - val_loss: 1.0209 - val_accuracy: 0.6298
Epoch 6/15
1302/1302 [==============================] - 86s 66ms/step - loss: 1.0050 - accuracy: 0.6507 - val_loss: 1.0166 - val_accuracy: 0.6363
Epoch 7/15
1302/1302 [==============================] - 85s 65ms/step - loss: 0.9859 - accuracy: 0.6638 - val_loss: 1.0163 -

**Translating new sentences with our RNN encoder and decoder**

In [25]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
I have to run.
[start] tengo que [UNK] [end]
-
There were few people in the park.
[start] había algunas personas en el parque [end]
-
I need to learn French.
[start] necesito aprender francés [end]
-
It's been a long time since we last saw each other.
[start] ha pasado mucho tiempo desde el tiempo [end]
-
You must know yourself.
[start] debes saber a ti mismo [end]
-
Tom doesn't even know how to start a lawn mower.
[start] tom ni siquiera sabe cómo [UNK] una clase de [UNK] [end]
-
Are they satisfied?
[start] están ellos [end]
-
I should've paid a little more attention.
[start] debería haber un poco más de la comida [end]
-
Tom hung out with his friends last night.
[start] tom anoche con sus amigos [end]
-
Tom wanted to buy a new pair of tennis shoes.
[start] tom quería comprar un par de zapatos por la noche de comprar [end]
-
We'll start when he gets here.
[start] cuando él te [UNK] aquí [end]
-
Tom and I have fun together.
[start] tom y yo nos juntos juntos [end]
-
Take your umbrell

Toy 모델을 개선하는 방법:

* 순환 레이어의 딥 스택(엔코더 및 디코더 모두)
* GRU 대신 LSTM 사용
* ...


제한 사항

* 소스 seq 표현은 인코더 상태(state) 벡터(크기, 소스 문장의 복잡성)에 유지되어야 합니다.
* RNN은 점진적으로 과거를 잊어버립니다. 긴 시퀀스에 적합하지 않음

Solution: Transformers

### Sequence-to-sequence learning with Transformer

Neural attention: RNN이 처리할 수 있는 것보다 훨씬 더 길고 복잡한 시퀀스

**Transformer Incoder** 는 소스 시퀀스를 읽고 인코딩된 표현을 생성합니다.

RNN 인코더와 달리 Transformer 인코더는 인코딩된 표현을 시퀀스 형식으로 유지합니다. 이는 컨텍스트 인식(context-aware) 임베딩 벡터의 시퀀스입니다.

**Transformer Decoder**: RNN 디코더와 마찬가지로 대상 시퀀스에서 토큰 0...N을 읽고 토큰 N+1을 예측하기 위해 연결합니다.

하지만: Neural attention를 사용하여 인코딩된 소스 문장에서 현재 예측하려는 대상 토큰과 가장 밀접하게 관련된 토큰을 식별합니다.

<img src="https://drek4537l1klr.cloudfront.net/chollet2/Figures/11-14.png" width="500"><p style="text-align:center">Figure 11.14 The TransformerDecoder is similar to the TransformerEncoder, except it features an additional attention block where the keys and values are the source sequence encoded by the TransformerEncoder. Together, the encoder and the decoder form an end-to-end Transformer.</p>


<img src="https://pytorch.org/tutorials/_images/transformer_architecture.jpg" width="500">

<img src="https://jalammar.github.io/images/t/transformer_decoding_2.gif">

#### The Transformer decoder

**The `TransformerDecoder`**

In [26]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

<img src="https://sshleifer.github.io/blog_v2/images/copied_from_nb/seq2seq_dec.png" width="200">

#### Putting it all together: A Transformer for machine translation

**PositionalEmbedding layer**

In [27]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**End-to-end Transformer**

In [28]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

**Training the sequence-to-sequence Transformer**

In [29]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 76s 57ms/step - loss: 1.7063 - accuracy: 0.4056 - val_loss: 1.3620 - val_accuracy: 0.4967
Epoch 2/30
1302/1302 [==============================] - 75s 57ms/step - loss: 1.3737 - accuracy: 0.5179 - val_loss: 1.2153 - val_accuracy: 0.5483
Epoch 3/30
1302/1302 [==============================] - 77s 59ms/step - loss: 1.2161 - accuracy: 0.5707 - val_loss: 1.1035 - val_accuracy: 0.5915
Epoch 4/30
1302/1302 [==============================] - 78s 60ms/step - loss: 1.1189 - accuracy: 0.6053 - val_loss: 1.0640 - val_accuracy: 0.6135
Epoch 5/30
1302/1302 [==============================] - 78s 60ms/step - loss: 1.0636 - accuracy: 0.6299 - val_loss: 1.0240 - val_accuracy: 0.6306
Epoch 6/30
1302/1302 [==============================] - 79s 60ms/step - loss: 1.0287 - accuracy: 0.6474 - val_loss: 1.0114 - val_accuracy: 0.6380
Epoch 7/30
1302/1302 [==============================] - 79s 61ms/step - loss: 1.0034 - accuracy: 0.6618 - val_loss: 1.0000 -

**Translating new sentences with our Transformer model**

In [30]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Who wants a drink?
[start] quién quiere una paz [end]
-
"What's happening in the cave? I'm curious." "I have no idea."
[start] qué ha salido de la [UNK] no tengo ni idea de salir [end]
-
He didn't attend the meeting.
[start] no [UNK] [end]
-
I believe that the boy is honest.
[start] creo que el niño es honesto [end]
-
It looks like this car is his.
[start] parece que este coche es mary [end]
-
Tom asked Mary to warm up some leftovers.
[start] tom le pidió a mary que se [UNK] algunos de los ojos [end]
-
Tom managed to tell me the truth.
[start] tom me pidió que fuera la verdad [end]
-
He's depressed.
[start] está [UNK] [end]
-
I ran home.
[start] me encontré a la casa [end]
-
The result was really satisfying.
[start] el resultado fue realmente [UNK] [end]
-
She failed to appear.
[start] ella no Él [UNK] [end]
-
You can't park here. However, there is a parking lot just around the corner.
[start] no puedes miedo por nadie [UNK] en un [UNK] por [UNK] sobre la isla [end]
-
Don't forget to

## Summary

* 두 종류의 NLP 모델
   - back-of-words 모델: 단어 세트 또는 N-grams (Dense Layers)
   - 시퀀스 모델: 단어 순서(RNN, 1D convnet 또는 Transformer)
   
   
* 텍스트 분류: `#samples / mean # words er sample` 을 사용하면 back-of-words 모델을 사용해야 하는지, 시퀀스 모델을 사용해야 하는지 결정하는 데 도움이 됩니다.


* Sequence-to-sequence 학습은 기계 번역을 포함한 많은 NLP 문제를 해결하는 데 적용할 수 있는 일반적이고 강력한 학습 프레임워크입니다.
   - 인코더: 소스 시퀀스를 처리
   - 디코더: 과거 토큰과 인코더 처리된 소스 시퀀스를 보고 타겟 시퀀스에서 미래 토큰을 예측합니다.
   
   
* Neural attention: 문맥 인식 단어 표현을 만드는 방법. Transformer 아키텍처의 기초입니다.


* Transformer 아키텍처(TransformerEncoder, TransformerDecoder)는 sequence-to-sequence 작업에서 우수한 결과를 제공합니다. TransformerEncoder는 텍스트 분류 또는 모든 종류의 단일 입력 NLP 작업에도 사용할 수 있습니다.
